# Crear dataset de entrada para predecir

In [63]:
import pandas as pd
import os
from unidecode import unidecode
import Levenshtein 
from difflib import get_close_matches

#### Leer esxcell mercado/plantilla

In [64]:
import pandas as pd

# Ruta al archivo Excel
archivo_excel = "../mi_plantilla2023-12-27--18-45-S.xlsx"

# Lee el archivo Excel con pandas y especifica que no hay encabezado
df = pd.read_excel(archivo_excel, header=None)

# Obtén los valores de la primera columna (columna 0)
valores_Mercado = df.iloc[:, 0].tolist()

# Imprime o utiliza los valores según sea necesario
print(valores_Mercado)
print(len(valores_Mercado))


['G. Mamardashvili', 'A. Pedraza', 'A. Rüdiger', 'E. García', 'Y. Couto', 'A. Baena', 'A. Tchouaméni', 'F. De Jong', 'M. Llorente', 'N. Maksimović', 'P. Torre', 'V. Tsygankov', 'Á. Morata', 'A. Douvikas', 'C. Ramos']
15


#### Función para encontrar los jugaodres de mi plantilla/mercado en el dataset de estadisticas

In [65]:
def encontrar_coincidencias(valores_Historico, valores_Mercado):
    print("Buscando coincidencias...\n")
    posiciones = []

    for pos, nombre_completo in enumerate(valores_Historico):
        coincidencias_iniciales = [nombre_abreviado for nombre_abreviado in valores_Mercado if nombre_abreviado.startswith(nombre_completo[0])]

        coincidencias_texto = get_close_matches(nombre_completo, coincidencias_iniciales)

        if coincidencias_texto:
            posiciones.append(pos)
            print(f"Coincidencia en {pos} para '{nombre_completo}': {coincidencias_texto[0]}")
    print("\n" + "." * 150+ "\n")
    return posiciones

#### Buscar jugaodres en los datasets de estadisticas que me interesan (jugaodres de mi plantilla / jugaodres en el mercado actual)

In [73]:
# Ruta a la carpeta que contiene los archivos Excel
carpeta_excel = "../data/test_precedecir"

# Lista global para almacenar todas las filas seleccionadas
filas_jugadores = []

# Lista todos los archivos en la carpeta con extensión .xlsx
archivos_excel = [archivo for archivo in os.listdir(carpeta_excel) if archivo.endswith(".xlsx")]

# Itera sobre cada archivo Excel en la carpeta
for archivo in archivos_excel:
    
    print("\n" + "=" * 150 + "\n")  # Separador visual entre archivos
    
    # Ruta al archivo Excel actual
    ruta_archivo = os.path.join(carpeta_excel, archivo)

    # Lee el archivo Excel con pandas y especifica que no hay encabezado
    df = pd.read_excel(ruta_archivo, header=None)

    # Obtén los valores de la primera columna (o cualquier columna que desees)
    valores_Historico = df.iloc[:, 0].tolist()

    # Llama a la función encontrar_coincidencias y actualiza los valores de los índices
    posiciones_coincidentes = encontrar_coincidencias(valores_Historico, valores_Mercado)
    
    # Accede a las filas correspondientes a los índices proporcionados
    filas_seleccionadas = df.iloc[posiciones_coincidentes]

    # Convierte las filas seleccionadas a una lista de diccionarios para un fácil acceso a los atributos
    datos_filas = filas_seleccionadas.to_dict(orient='records')

    # Agrega las filas seleccionadas a la lista global
    global filas_jugadores
    filas_jugadores.extend(datos_filas)

    # Imprime los datos de las filas seleccionadas
    for idx, fila in zip(posiciones_coincidentes, datos_filas):
        print(f"Coincidencia en {ruta_archivo}, fila {idx}: {fila}\n")

# Ahora, filas_jugadores contiene todas las filas seleccionadas de todos los archivos Excel



Buscando coincidencias...

Coincidencia en 120 para 'Anastasios Douvikas': A. Douvikas
Coincidencia en 125 para 'Adrià Pedrosa': A. Pedraza
Coincidencia en 148 para 'Nemanja Maksimović': N. Maksimović
Coincidencia en 162 para 'Chris Ramos': C. Ramos
Coincidencia en 210 para 'Álvaro Morata': Á. Morata
Coincidencia en 221 para 'Marcos Llorente': M. Llorente
Coincidencia en 247 para 'Eric García': E. García
Coincidencia en 249 para 'Yan Couto': Y. Couto
Coincidencia en 267 para 'Viktor Tsygankov': V. Tsygankov
Coincidencia en 274 para 'Pablo Torre': P. Torre
Coincidencia en 288 para 'Antonio Rüdiger': A. Rüdiger
Coincidencia en 324 para 'Aurélien Tchouaméni': A. Tchouaméni
Coincidencia en 374 para 'Frenkie De Jong': F. De Jong
Coincidencia en 380 para 'Giorgi Mamardashvili': G. Mamardashvili

......................................................................................................................................................

Coincidencia en ../data/test_precedecir\datas

In [105]:
nombres_vistos = set()

for indice, fila_jugador in enumerate(filas_jugadores):
    nombre_jugador = fila_jugador[0]

    # Verifica si el nombre del jugador ya ha sido visto
    if nombre_jugador in nombres_vistos:
        print(f"El nombre {nombre_jugador} está duplicado en la lista en el índice {indiceO} :: {indice}.")
        # Puedes realizar alguna acción adicional si encuentras un duplicado
    else:
        # Agrega el nombre del jugador al conjunto de nombres vistos
        nombres_vistos.add(nombre_jugador)

El nombre Anastasios Douvikas está duplicado en la lista en el índice 28 :: 14.
El nombre Adrià Pedrosa está duplicado en la lista en el índice 28 :: 15.
El nombre Nemanja Maksimović está duplicado en la lista en el índice 28 :: 16.
El nombre Chris Ramos está duplicado en la lista en el índice 28 :: 17.
El nombre Álvaro Morata está duplicado en la lista en el índice 28 :: 18.
El nombre Marcos Llorente está duplicado en la lista en el índice 28 :: 19.
El nombre Eric García está duplicado en la lista en el índice 28 :: 20.
El nombre Yan Couto está duplicado en la lista en el índice 28 :: 21.
El nombre Viktor Tsygankov está duplicado en la lista en el índice 28 :: 22.
El nombre Pablo Torre está duplicado en la lista en el índice 28 :: 23.
El nombre Antonio Rüdiger está duplicado en la lista en el índice 28 :: 24.
El nombre Aurélien Tchouaméni está duplicado en la lista en el índice 28 :: 25.
El nombre Frenkie De Jong está duplicado en la lista en el índice 28 :: 26.
El nombre Giorgi Mamar